# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_weather=pd.read_csv("output_data/city_weather.csv")
city_weather.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# max temp lower than 80 degrees but higher than 70 , wind speed less than 10 mph, zero cloudiness drop all rows that dont contain these

# Store latitude and longitude in locations
temps = city_weather.loc[(city_weather["Max_Temp"]<=80) & (city_weather["Max_Temp"]>=70)]
winds=temps.loc[temps["Wind_Speed"]<=10]
clouds= winds.loc[winds["Cloudiness"]==0]
locations=clouds["Lat","Long"]

# # Fill NaN values and convert to float
rating = clouds["Humidity"].astype(float)


In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
temps = city_weather.loc[(city_weather["Max_Temp"]<=80) & (city_weather["Max_Temp"]>=70)]
wind=city_weather.loc[temps["Wind_Speed"]<=10]
clouds= city_weather.loc[winds["Cloudiness"]==0]

data_params[temps&wind&clouds]

city_data_params=city_weather[data_params]

city_data_params.to_csv("city_data_params",index=False)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
city_data_params=hotel_df
hotel_df["Hotel_Name"] = ""

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:
params = {
    "radius":"5000",
    "keyword": "Hotel",
    "key": gkey,
}
for index, row in types_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    params['location'] = lat&lng
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel'] = results[0]['hotel']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(hotel_df["Hotel_Name"])
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Display figure
fig.add_layer(markers,heat_layer)
fig